In [1]:
import pandas as pd 
import bz2
import pickle
import os
import shutil
import json
import tqdm
import multiprocessing

In [2]:
path = '/home/rgopikrishna/vrif/data/newomnisig-kop-*_8apr21to8jun21'

In [3]:
len(os.listdir(path)),os.listdir(path)[0],os.listdir(path)[len(os.listdir(path))-1]

(29280,
 '2021-04-08-04-00-00_2021-04-08-04-03-00.pkl.bz2',
 '2021-06-08-03-57-00_2021-06-08-04-00-00.pkl.bz2')

In [5]:
zfile = bz2.BZ2File('/home/rgopikrishna/vrif/data/newomnisig-kop-*_8apr21to8jun21/2021-06-08-03-57-00_2021-06-08-04-00-00.pkl.bz2','rb')
data = pickle.load(zfile)

In [6]:
data[0]

{'_index': 'newomnisig-kop-000038',
 '_type': '_doc',
 '_id': '1sDG6XkBsljR9TZ_PFoC',
 '_score': 0.0,
 '_source': {'agent': {'hostname': 'xavier1',
   'type': 'omnisig',
   'version': '2.2.0'},
  'process': {'name': 'omnisig'},
  'message': 'FM signal detected',
  'radio': {'sample_rate': 40000000.0,
   'center_frequency': 820000000.0,
   'gain': 32.0},
  'tags': ['omnisig', 'FM'],
  '@timestamp': '2021-06-08T03:59:12.460316Z',
  'ecs': {'version': '1.5'},
  'host': {'geo': {'location': {'lon': -75.39054870605469,
     'lat': 40.11270523071289},
    'altitude_meters': 50.400001525878906},
   'hostname': 'xavier1',
   'name': 'xavier1'},
  'event': {'duration': 1638400,
   'kind': 'event',
   'module': 'omnisig',
   'type': 'info',
   'dataset': 'omnisig.detector'},
  'signal': {'date': 1623124752460,
   'hour_utc': 3,
   'rssi': -31.402961240158746,
   'hour_local': 23,
   'bandwidth': 20360.74642086029,
   'energy_estimate': 14.740002971993801,
   'confidence': 0.8526962399482727,
   

In [7]:
def extractfmp25ltefrom3minfile(file):
    zfile = bz2.BZ2File(path+'/'+file,'rb')
    data = pickle.load(zfile)
    
    filtereddatafm = []
    filtereddatap25 = []
    filtereddatalte = []
    
    for idx,pkt in enumerate(data): 
        if 'LTE' in pkt['_source']['tags']:
            filtereddatalte.append(pkt)
        if 'FM' in pkt['_source']['tags']:
            filtereddatafm.append(pkt)
        if 'P25' in pkt['_source']['tags']:
            filtereddatap25.append(pkt)

    with open('/home/rgopikrishna/vrif/data/newomnisig-kop-*_8apr21to8jun21_signalsseperated/lte_8apr21to8jun21/'+file.split('.pkl.bz2')[0]+'.pkl',"wb") as f:
        pickle.dump(filtereddatalte,f)
    with open('/home/rgopikrishna/vrif/data/newomnisig-kop-*_8apr21to8jun21_signalsseperated/fm_8apr21to8jun21/'+file.split('.pkl.bz2')[0]+'.pkl',"wb") as f:
        pickle.dump(filtereddatafm,f)
    with open('/home/rgopikrishna/vrif/data/newomnisig-kop-*_8apr21to8jun21_signalsseperated/p25_8apr21to8jun21/'+file.split('.pkl.bz2')[0]+'.pkl',"wb") as f:
        pickle.dump(filtereddatap25,f)

In [8]:
%%time
p = multiprocessing.Pool(multiprocessing.cpu_count())
p.map(extractfmp25ltefrom3minfile,os.listdir(path))

CPU times: user 959 ms, sys: 975 ms, total: 1.93 s
Wall time: 5min 18s


In [10]:
print(len(os.listdir('/home/rgopikrishna/vrif/data/newomnisig-kop-*_8apr21to8jun21_signalsseperated/lte_8apr21to8jun21')))
print(len(os.listdir('/home/rgopikrishna/vrif/data/newomnisig-kop-*_8apr21to8jun21_signalsseperated/fm_8apr21to8jun21')))
print(len(os.listdir('/home/rgopikrishna/vrif/data/newomnisig-kop-*_8apr21to8jun21_signalsseperated/p25_8apr21to8jun21')))

29280
29280
29280


In [11]:
for band in ['lte','fm','p25']:
    path = '/home/rgopikrishna/vrif/data/newomnisig-kop-*_8apr21to8jun21_signalsseperated/{}_8apr21to8jun21'.format(band)
    print(len(os.listdir(path)))
    print(path)
    print(os.listdir(path)[:10])

    packets = []
    for idx,file in tqdm.tqdm(enumerate(os.listdir(path))):
        with open(path+'/'+file,'rb') as f:
            data = pickle.load(f)
        for pkt in data:
            packets.append({
                'timestamp':pkt['_source']['@timestamp'],
                'frequency':pkt['_source']['signal']['center_frequency'],
                'bandwidth':pkt['_source']['signal']['bandwidth'],
                'rssi':pkt['_source']['signal']['rssi'],
                'tag':pkt['_source']['tags']})

    with open('/home/rgopikrishna/vrif/data/newomnisig-kop-*_8apr21to8jun21_signalsseperated/filteredpackets_{}_8apr21to8jun21.json'.format(band),'a') as file:
        for item in packets:
            file.write(json.dumps(item)+"\n")

26it [00:00, 254.78it/s]

29280
/home/rgopikrishna/vrif_working/data/newomnisig-kop-*_8apr21to8jun21_signalsseperated/lte_8apr21to8jun21
['2021-04-09-07-27-00_2021-04-09-07-30-00.pkl', '2021-04-08-22-18-00_2021-04-08-22-21-00.pkl', '2021-04-08-04-00-00_2021-04-08-04-03-00.pkl', '2021-04-11-05-12-00_2021-04-11-05-15-00.pkl', '2021-04-10-20-03-00_2021-04-10-20-06-00.pkl', '2021-04-16-04-09-00_2021-04-16-04-12-00.pkl', '2021-04-21-21-24-00_2021-04-21-21-27-00.pkl', '2021-04-13-21-15-00_2021-04-13-21-18-00.pkl', '2021-04-22-15-42-00_2021-04-22-15-45-00.pkl', '2021-04-15-19-00-00_2021-04-15-19-03-00.pkl']


29280it [03:28, 140.68it/s]


29280
/home/rgopikrishna/vrif_working/data/newomnisig-kop-*_8apr21to8jun21_signalsseperated/fm_8apr21to8jun21
['2021-04-09-07-27-00_2021-04-09-07-30-00.pkl', '2021-04-08-22-18-00_2021-04-08-22-21-00.pkl', '2021-04-11-05-12-00_2021-04-11-05-15-00.pkl', '2021-04-08-04-00-00_2021-04-08-04-03-00.pkl', '2021-04-21-21-24-00_2021-04-21-21-27-00.pkl', '2021-04-10-20-03-00_2021-04-10-20-06-00.pkl', '2021-04-16-04-09-00_2021-04-16-04-12-00.pkl', '2021-04-22-15-42-00_2021-04-22-15-45-00.pkl', '2021-04-13-21-15-00_2021-04-13-21-18-00.pkl', '2021-04-15-19-00-00_2021-04-15-19-03-00.pkl']


29280it [02:49, 172.82it/s]


29280
/home/rgopikrishna/vrif_working/data/newomnisig-kop-*_8apr21to8jun21_signalsseperated/p25_8apr21to8jun21
['2021-04-09-07-27-00_2021-04-09-07-30-00.pkl', '2021-04-08-22-18-00_2021-04-08-22-21-00.pkl', '2021-04-11-05-12-00_2021-04-11-05-15-00.pkl', '2021-04-08-04-00-00_2021-04-08-04-03-00.pkl', '2021-04-21-21-24-00_2021-04-21-21-27-00.pkl', '2021-04-10-20-03-00_2021-04-10-20-06-00.pkl', '2021-04-16-04-09-00_2021-04-16-04-12-00.pkl', '2021-04-22-15-42-00_2021-04-22-15-45-00.pkl', '2021-04-15-19-00-00_2021-04-15-19-03-00.pkl', '2021-04-13-21-15-00_2021-04-13-21-18-00.pkl']


29280it [11:38, 41.90it/s] 
